In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import math
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json
%config Completer.use_jedi = False  # Fixes autocomplete issues
%config InlineBackend.figure_format = 'retina'  # Improves plot resolution

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
from sleap import load_and_process as lp

In [ ]:
# data paths setup
#-------------------------------

#data_dir = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1')
data_dir = Path('/Volumes/RanczLab2/20241125_Cohort1_rotation/Visual_mismatch_day3')
#data_dir = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day4')
rawdata_paths = [Path(p) for p in data_dir.iterdir() if p.is_dir() and not p.name.endswith('_processedData')]

#-------------------------------
# initial variables setup
#-------------------------------
rawdata_path = rawdata_paths[0]
time_window_start = -2  # s, FOR PLOTTING PURPOSES
time_window_end = 10  # s, FOR PLOTTING PURPOSES
baseline_window = (-5, 0) # s, FOR baselining averages 
event_name = "Apply halt: 2s" 
vestibular_mismatch = False
common_resampled_rate = 1000 #in Hz
plot_fig1 = True
# for saccades
framerate = 59.77  # Hz (in the future, should come from saved data)
threshold = 65  # px/s FIXME make this adaptive
refractory_period = pd.Timedelta(milliseconds=100)  # msec, using pd.Timedelta for datetime index
plot_saccade_detection_QC = False

data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])


In [ ]:
# load downsampled data 
#-------------------------------
photometry_tracking_encoder_data = pd.read_parquet(data_path / "photometry_tracking_encoder_data.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(data_path / "camera_photodiode_data.parquet", engine="pyarrow")
experiment_events = pd.read_parquet(data_path / "experiment_events.parquet", engine="pyarrow")
photometry_info = pd.read_parquet(data_path / "photometry_info.parquet", engine="pyarrow")
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)

print(f"✅ Finished loading all parquet files")

# Calculate time differences between event_name events
event_times = experiment_events[experiment_events["Event"] == event_name].index
time_diffs = event_times.to_series().diff().dropna().dt.total_seconds()

# # Print the 5 shortest time differences
# print("5 shortest time differences between events:")
# print(time_diffs.nsmallest(5))

if (time_diffs < 10).any():
    print(f"⚠️ Warning: Some '{event_name}' events are less than 10 seconds apart. Consider applying a filter to events.")

mouse_name = process.check_exp_events(experiment_events, photometry_info, verbose = True)


In [ ]:
# PLOT FIGURE to ascertain everything is well loaded
#---------------------------------------------------

df_to_analyze = photometry_tracking_encoder_data["Photodiode_int"] #using downsampled values in common time grid 
#df_to_analyze = camera_photodiode_data["Photodiode"] #use async raw values if needed for troubleshooting, but the nearest indices needs to be found , see couple of lines below

if vestibular_mismatch or event_name == "No halt": #determine halt times based on experiment events 
    photodiode_halts = experiment_events[experiment_events["Event"] == event_name].index.tolist()
    nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
    photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices] # as experiment events timestamps are not in the same time grid as downsampled data
    print ("ℹ️ INFO: vestibular MM or 'No halt', no signal in the photodiode, using experiment events for MM times")
else: #determine exact halt times based on photodiode signal
    photodiode_halts, photodiode_delay_min, photodiode_delay_avg, photodiode_delay_max = process.analyze_photodiode(df_to_analyze, experiment_events, event_name, plot = True)
# nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
# photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
if plot_fig1:
    process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = True, show_figure = True, downsample_factor=50)
else: 
    print ("ℹ️ INFO: skipping figure 1")
del df_to_analyze
gc.collect()
None

## LOOK INTO THE superbig filtering in the velocity and ACC calculations (looks like it's 1 Hz???)

Saccade detection(from notebook SANDBOX_1_2)

In [ ]:
# #load already processed sleap data 
# # Load all CSV files present in the folder
# VideoData1 = (rawdata_path.parent / f"{rawdata_path.name}_processedData/Video_Sleap_Data1/")
# csv_files = [file for file in sleap_data_path.iterdir() if file.suffix == '.csv']

In [ ]:
df = photometry_tracking_encoder_data.copy()
print(df.columns)

### Plot all aligned data for individual halt events, the plot baselined averages ± sem

In [ ]:
# SACCADE DETECTION
#-------------------------------
# 0. align data
aligned_data = []

for halt_time in photodiode_halts:
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time + pd.Timedelta(seconds=time_window_start)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=time_window_end))
    ].copy()
    
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    window_data["Halt Time"] = halt_time
    aligned_data.append(window_data)

aligned_df = pd.concat(aligned_data, ignore_index=True)

aligned_df = aligned_df.copy()

# 1. Preprocess: smooth
aligned_df['X_smooth'] = (
    aligned_df['Ellipse.Center.X_eye1']
      .rolling(window=5, center=True)
      .median()
      .bfill()
      .ffill()
)
# 2. Compute instantaneous velocity
#   dt in seconds
aligned_df['dt'] = aligned_df['Time (s)'].diff()
#   vel = dX / dt
aligned_df['vel_x'] = aligned_df['X_smooth'].diff() / aligned_df['dt']

# --- Plot smoothed trace and on separate y axis plot velocity---
# Create subplots with shared x-axis for synchronized zooming
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,  # This ensures x-axis zoom synchronization
    vertical_spacing=0.1,
    subplot_titles=('X Position (px)', 'Velocity (px/s)')
)

# Add X_smooth to the first subplot
fig.add_trace(
    go.Scatter(
        x=aligned_df['Time (s)'],
        y=aligned_df['X_smooth'],
        mode='lines',
        name='Smoothed X',
        line=dict(color='blue')
    ),
    row=1, col=1
)

# Add velocity to the second subplot
fig.add_trace(
    go.Scatter(
        x=aligned_df['Time (s)'],
        y=aligned_df['vel_x'],
        mode='lines',
        name='Velocity',
        line=dict(color='red')
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title='Smoothed X and Velocity Traces (Synchronized Zoom)',
    height=600,  # Adjust height for two subplots
    showlegend=True,
    legend=dict(x=0.01, y=0.99)
)

# Update x-axes
fig.update_xaxes(title_text="Time (s)", row=2, col=1)

# Update y-axes
fig.update_yaxes(title_text="X Position (px)", row=1, col=1)
fig.update_yaxes(title_text="Velocity (px/s)", row=2, col=1)

fig.show()

In [ ]:


# --- 1. Build aligned_df with "Halt Time" ---
aligned_data = []
for halt_time in photodiode_halts:
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time + pd.Timedelta(seconds=time_window_start)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=time_window_end))
    ].copy()

    # Time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    # Add Halt Time column for grouping later
    window_data["Halt Time"] = halt_time

    aligned_data.append(window_data)

aligned_df = pd.concat(aligned_data, ignore_index=True)

# --- 2. Smooth position and compute velocity ---
aligned_df['X_smooth'] = (
    aligned_df['Ellipse.Center.X_eye1']
        .rolling(window=5, center=True)
        .median()
        .bfill()
        .ffill()
)
aligned_df['dt'] = aligned_df['Time (s)'].diff()
aligned_df['vel_x'] = aligned_df['X_smooth'].diff() / aligned_df['dt']

# --- 3. Saccade detection thresholds ---
abs_vel = aligned_df['vel_x'].abs().dropna()
k = 3
vel_thresh = abs_vel.mean() + k * abs_vel.std()
pos_thresh = vel_thresh
neg_thresh = -vel_thresh

aligned_df['is_sac_pos'] = aligned_df['vel_x'] > pos_thresh
aligned_df['is_sac_neg'] = aligned_df['vel_x'] < neg_thresh

aligned_df['sac_pos_id'] = (
    (aligned_df['is_sac_pos'] & ~aligned_df['is_sac_pos'].shift(fill_value=False))
    .cumsum()
    .where(aligned_df['is_sac_pos'], 0)
)
aligned_df['sac_neg_id'] = (
    (aligned_df['is_sac_neg'] & ~aligned_df['is_sac_neg'].shift(fill_value=False))
    .cumsum()
    .where(aligned_df['is_sac_neg'], 0)
)

# --- 4. Summarization function WITH Halt Time ---
def summarize(df, col_id, direction):
    return (
        df[df[col_id] > 0]
        .groupby(['Halt Time', col_id], as_index=False)
        .agg(
            start_time=('Time (s)', 'first'),
            end_time=('Time (s)', 'last'),
            peak_vel=('vel_x', lambda x: x.max() if direction == 'pos' else x.min()),
            amplitude=('X_smooth', lambda x: (x.max()-x.min()) if direction == 'pos' else (x.min()-x.max()))
        )
        .assign(direction=direction)
    )

sac_pos = summarize(aligned_df, 'sac_pos_id', 'pos')
sac_neg = summarize(aligned_df, 'sac_neg_id', 'neg')

saccades = pd.concat([sac_pos, sac_neg], ignore_index=True)

# --- 5. Apply refractory period PER halt ---
refractory_period = 0.100  # seconds

def keep_with_refractory(group):
    last_end = -float('inf')
    kept = []
    for _, row in group.sort_values('start_time').iterrows():
        if row['start_time'] - last_end >= refractory_period:
            kept.append(row.to_dict())
            last_end = row['end_time']
    return pd.DataFrame(kept)

filtered_saccades = (
    saccades
    .sort_values(['Halt Time', 'start_time'])
    .groupby('Halt Time', group_keys=False)
    .apply(keep_with_refractory)
    .reset_index(drop=True)
)

print(f"After refractory period: {len(filtered_saccades)} saccades remain")
print(filtered_saccades.head())


In [ ]:
# --- Subplot Grid Setup ---
n_events = len(photodiode_halts)
n_cols = 4
n_rows = math.ceil(n_events / n_cols)

# Create subplots
specs = [[{} for _ in range(n_cols)] for _ in range(n_rows)]
# subplot_titles = [f'Event: {halt_time}' for halt_time in photodiode_halts]
fig = sp.make_subplots(
    rows=n_rows, 
    cols=n_cols, 
    # subplot_titles=subplot_titles, 
    specs=specs
)
base_extra = n_events + 1  # base index for extra y-axes

for i, halt_time in enumerate(photodiode_halts):
    row = (i // n_cols) + 1
    col = (i % n_cols) + 1
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]

    # Fluorescence traces on primary y-axis
    fig.add_trace(
        go.Scatter(
            x=subset["Time (s)"],
            y=subset["z_470"],
            mode='lines',
            name='z_470',
            line=dict(color='green'),
            showlegend=(i == 0)
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=subset["Time (s)"],
            y=subset["z_560"],
            mode='lines',
            name='z_560',
            line=dict(color='red'),
            showlegend=(i == 0)
        ),
        row=row, col=col
    )
    fig.update_yaxes(title_text="Fluorescence z-score", row=row, col=col)

    # Add saccade shaded regions
    halt_saccades = filtered_saccades[filtered_saccades["Halt Time"] == halt_time]
    for _, sac in halt_saccades.iterrows():
        color = "orange" if sac['direction'] == 'pos' else "blue"
        fig.add_vrect(
            x0=sac['start_time'],
            x1=sac['end_time'],
            fillcolor=color,
            opacity=0.9,
            line_width=1,
            line_color=color,
            row=row, col=col
        )

    # Extra y-axis for Ellipse Center X Trace
    extra_axis_number = base_extra + i
    extra_axis_trace_ref = f"y{extra_axis_number}"

    fig.add_trace(
        go.Scatter(
            x=subset["Time (s)"],
            y=subset["Ellipse.Diameter_eye1"],
            mode='lines',
            name='Ellipse Diameter X',
            line=dict(color='purple'),
            showlegend=(i == 0)
        ),
        row=row, col=col
    )
    fig.data[-1].update(yaxis=extra_axis_trace_ref)

    # Define extra y-axis layout for this subplot
    xaxis_number = (row - 1) * n_cols + col
    x_anchor = "x" if xaxis_number == 1 else f"x{xaxis_number}"
    primary_y = "y" if xaxis_number == 1 else f"y{xaxis_number}"
    extra_axis_layout_key = f"yaxis{extra_axis_number}"
    fig.update_layout(
        {extra_axis_layout_key: dict(
            title="pupil diameter (px)",
            overlaying=primary_y,
            anchor=x_anchor,
            side="right",
            position=0.95,
            showgrid=False
        )}
    )

# Update common axis labels
fig.update_xaxes(title_text="Time (s)")
    # Update y-axis title for fluorescence z-score and adjust subplot spacing
fig.update_layout(
        height=300 * n_rows,
        width=300 * n_cols,
        title_text="Fluorescence & Ellipse Center X for Each Event",
        template='plotly_white',
        font=dict(size=8),
        margin=dict(l=50, r=50, t=50, b=50))
fig.show()
gc.collect()


In [ ]:

# #selected_indices = [0, 1, 11, 16]
selected_indices = list(range(len(photodiode_halts)))
# # Create an empty list to store aligned data
aligned_data = []

# Loop through each halt event time using enumerate
for idx, halt_time in enumerate(photodiode_halts):
    if idx not in selected_indices:
        continue  # Skip halt times that are not in our selected indices
    
    # Extract data within the selected time window
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time + pd.Timedelta(seconds=time_window_start)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=time_window_end))
    ].copy()

    # Compute time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    
    # Add event identifier
    window_data["Halt Time"] = halt_time

    # Store aligned data
    aligned_data.append(window_data)

# Concatenate all windows into a single DataFrame
aligned_df = pd.concat(aligned_data, ignore_index=True)


# Compute mean and standard error of the mean (SEM)
mean_df = aligned_df.groupby("Time (s)").mean()
sem_df = aligned_df.groupby("Time (s)").sem()  

# Create figure for the two plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=True, gridspec_kw={'wspace': 0.3, 'hspace': 0.3})
plt.rcParams.update({'font.size': 8})

### PLOT 1: Individual Traces - Photodiode, z_470, z_560 ###
ax1 = axes[0]

for halt_time in photodiode_halts:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1.plot(subset["Time (s)"], subset["Photodiode_int"], color='grey', alpha=0.5)

ax1.set_xlabel('Time (s) relative to halt')
ax1.set_ylabel('Photodiode')
ax1.set_title('Photodiode, z_470, and z_560')

ax1_2 = ax1.twinx()
for halt_time in photodiode_halts:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1_2.plot(subset["Time (s)"], subset["z_470"], color='green', linestyle='-', alpha=0.5)
    ax1_2.plot(subset["Time (s)"], subset["z_560"], color='red', linestyle='-', alpha=0.5)

ax1_2.set_ylabel('Fluorescence (z-score)', color='green')

### PLOT 2: Mean & SEM of All Signals ###
ax2 = axes[1]

# Photodiode
ax2.plot(mean_df.index, mean_df["Photodiode_int"], color='grey', alpha=0.8)
ax2.fill_between(mean_df.index, mean_df["Photodiode_int"] - sem_df["Photodiode_int"], 
                 mean_df["Photodiode_int"] + sem_df["Photodiode_int"], color='grey', alpha=0.2)

ax2.set_xlabel('Time (s) relative to halt')
ax2.set_ylabel('Photodiode')
ax2.set_title('Mean & SEM of All Signals')

# Fluorescence signals (z_470 and z_560)
ax2_2 = ax2.twinx()
ax2_2.plot(mean_df.index, mean_df["z_470"], color='green', linestyle='-', alpha=0.8)
ax2_2.fill_between(mean_df.index, mean_df["z_470"] - sem_df["z_470"], 
                   mean_df["z_470"] + sem_df["z_470"], color='green', alpha=0.2)

ax2_2.plot(mean_df.index, mean_df["z_560"], color='red', linestyle='-', alpha=0.8)
ax2_2.fill_between(mean_df.index, mean_df["z_560"] - sem_df["z_560"], 
                   mean_df["z_560"] + sem_df["z_560"], color='red', alpha=0.2)

ax2_2.set_ylabel('Fluorescence (z-score)', color='green')

# # Motor Velocity
# ax2_3 = ax2.twinx()
# ax2_3.spines['right'].set_position(('outward', 50))  
# ax2_3.plot(mean_df.index, mean_df["Motor_Velocity"], color='#00008B', linestyle='-', alpha=0.8)
# ax2_3.fill_between(mean_df.index, mean_df["Motor_Velocity"] - sem_df["Motor_Velocity"], 
#                    mean_df["Motor_Velocity"] + sem_df["Motor_Velocity"], color='#00008B', alpha=0.2)
# ax2_3.set_ylabel('Motor Velocity (Dark Blue)')
# ax2_3.yaxis.label.set_color('#00008B')

# # Running Velocity (Velocity_0X)
# ax2_4 = ax2.twinx()
# ax2_4.spines['right'].set_position(('outward', 100))  
# ax2_4.plot(mean_df.index, mean_df["Velocity_0X"]*1000, color='orange', linestyle='-', alpha=0.8)
# ax2_4.fill_between(mean_df.index, (mean_df["Velocity_0X"] - sem_df["Velocity_0X"])*1000, 
#                    (mean_df["Velocity_0X"] + sem_df["Velocity_0X"])*1000, color='orange', alpha=0.2)
# ax2_4.set_ylabel('Running velocity (mm/s²) WRONG SCALE?', color='orange')

# Turning Velocity (Velocity_0Y)
ax2_3 = ax2.twinx()
ax2_3.spines['right'].set_position(('outward', 150))  
ax2_3.plot(mean_df.index, mean_df["Velocity_0Y"], color='#4682B4', linestyle='-', alpha=0.8)
ax2_3.fill_between(mean_df.index, mean_df["Velocity_0Y"] - sem_df["Velocity_0Y"], 
                   mean_df["Velocity_0Y"] + sem_df["Velocity_0Y"], color='#4682B4', alpha=0.2)
ax2_3.set_ylabel('Turning velocity (deg/s²) WRONG SCALE?', color='#4682B4')

# ---- New code added below for Ellipse signals ----

# Ellipse.Diameter
ax2_4 = ax2.twinx()
ax2_4.spines['right'].set_position(('outward', 100))  
ax2_4.plot(mean_df.index, mean_df["Ellipse.Diameter_eye1"], color='purple', linestyle='-', alpha=0.8)
ax2_4.fill_between(mean_df.index, mean_df["Ellipse.Diameter_eye1"] - sem_df["Ellipse.Diameter_eye1"], 
                   mean_df["Ellipse.Diameter_eye1"] + sem_df["Ellipse.Diameter_eye1"], color='purple', alpha=0.2)
ax2_4.set_ylabel('Pupil Diameter', color='purple')

# Ellipse.Diameter
ax2_5 = ax2.twinx()
ax2_5.spines['right'].set_position(('outward', 50))  
ax2_5.plot(mean_df.index, mean_df["Ellipse.Center.X_eye1"], color='magenta', linestyle='-', alpha=0.8)
ax2_5.fill_between(mean_df.index, mean_df["Ellipse.Center.X_eye1"] - sem_df["Ellipse.Center.X_eye1"], 
                   mean_df["Ellipse.Center.X_eye1"] + sem_df["Ellipse.Center.X_eye1"], color='magenta', alpha=0.2)
ax2_5.set_ylabel('Ellipse Center X', color='magenta')

# Saccade Mean and SEM
# Compute mean and SEM for the number of saccades
mean_saccade_count = filtered_saccades.groupby("Halt Time").size().mean()
sem_saccade_count = filtered_saccades.groupby("Halt Time").size().sem()

# Plot the average distribution of the filtered saccades with a small bin size
ax2_6 = ax2.twinx()
ax2_6.spines['right'].set_position(('outward', 200))  

# Define bin size and create bins
bin_size = 1  # Small bin size
bins = np.arange(filtered_saccades["start_time"].min(), filtered_saccades["end_time"].max() + bin_size, bin_size)

# Compute histogram for saccade distribution
saccade_hist, bin_edges = np.histogram(filtered_saccades["start_time"], bins=bins)

# Compute bin centers
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the histogram as a line plot
ax2_6.plot(bin_centers, saccade_hist, color='cyan', linestyle='-', alpha=0.8)
ax2_6.set_ylabel('Saccade Distribution (cyan)', color='cyan')
ax2_6.set_xlabel('Time (s)')

# Adjust layout and show plot
fig.tight_layout()
plt.show()

del aligned_data
gc.collect()
None



In [ ]:
#baselining and plotting

# Set the plot width (in inches)
plot_width = 14  # Change this value to adjust the plot width

# Compute baseline values for each signal (excluding Photodiode)
baseline_df = aligned_df[(aligned_df["Time (s)"] >= baseline_window[0]) & 
                         (aligned_df["Time (s)"] <= baseline_window[1])].groupby("Halt Time").mean()

# Subtract baseline from each signal (except Photodiode)
for signal_name in ["z_470", "z_560", "Motor_Velocity", "Velocity_0X", "Velocity_0Y"]:
    aligned_df[f"{signal_name}_Baseline"] = aligned_df[signal_name] - aligned_df["Halt Time"].map(baseline_df[signal_name])

# ---- NEW: Baseline correction for Ellipse signals ----
for signal_name in ["Ellipse.Diameter_eye1", "Ellipse.Center.X_eye1"]:
    aligned_df[f"{signal_name}_Baseline"] = aligned_df[signal_name] - aligned_df["Halt Time"].map(baseline_df[signal_name])

# Compute mean and SEM of baseline-adjusted signals
mean_baseline_df = aligned_df.groupby("Time (s)").mean()
sem_baseline_df = aligned_df.groupby("Time (s)").sem()

# Function to ensure zero is centered while covering SEM values
def get_symmetric_ylim(mean_data, sem_data):
    max_abs_value = max(abs(mean_data).max() + sem_data.max(), abs(mean_data).min() - sem_data.min())
    return (-max_abs_value, max_abs_value)

# Create figure for the new baseline-corrected plot using plot_width parameter
fig, ax = plt.subplots(figsize=(plot_width, 6))

### PLOT: Mean & SEM of Baseline-Corrected Signals ###
ax.plot(mean_baseline_df.index, mean_baseline_df["Photodiode_int"], color='grey', alpha=0.8)
ax.fill_between(mean_baseline_df.index, mean_baseline_df["Photodiode_int"] - sem_baseline_df["Photodiode_int"], 
                mean_baseline_df["Photodiode_int"] + sem_baseline_df["Photodiode_int"], color='grey', alpha=0.2)

ax.set_xlabel('Time (s) relative to halt')
ax.set_ylabel('Photodiode', color='grey')
ax.set_title(f'Baselined Mean & SEM of All Signals - {mouse_name}')

# Fluorescence axis (z_470 and z_560)
ax2 = ax.twinx()
ax2.plot(mean_baseline_df.index, mean_baseline_df["z_470_Baseline"], color='green', linestyle='-', alpha=0.8)
ax2.fill_between(mean_baseline_df.index, mean_baseline_df["z_470_Baseline"] - sem_baseline_df["z_470_Baseline"], 
                 mean_baseline_df["z_470_Baseline"] + sem_baseline_df["z_470_Baseline"], color='green', alpha=0.2)
ax2.plot(mean_baseline_df.index, mean_baseline_df["z_560_Baseline"], color='red', linestyle='-', alpha=0.8)
ax2.fill_between(mean_baseline_df.index, mean_baseline_df["z_560_Baseline"] - sem_baseline_df["z_560_Baseline"], 
                 mean_baseline_df["z_560_Baseline"] + sem_baseline_df["z_560_Baseline"], color='red', alpha=0.2)
ax2.set_ylabel('Fluorescence (z-score, red 560nm)', color='green')
ax2.set_ylim(get_symmetric_ylim(
    pd.concat([mean_baseline_df["z_470_Baseline"], mean_baseline_df["z_560_Baseline"]]),
    pd.concat([sem_baseline_df["z_470_Baseline"], sem_baseline_df["z_560_Baseline"]])
))
ax2.yaxis.label.set_color('green')

# Motor Velocity axis
ax3 = ax.twinx()
ax3.spines['right'].set_position(('outward', 50))  
ax3.plot(mean_baseline_df.index, mean_baseline_df["Motor_Velocity_Baseline"], color='#00008B', linestyle='-', alpha=0.8)
ax3.fill_between(mean_baseline_df.index, mean_baseline_df["Motor_Velocity_Baseline"] - sem_baseline_df["Motor_Velocity_Baseline"], 
                 mean_baseline_df["Motor_Velocity_Baseline"] + sem_baseline_df["Motor_Velocity_Baseline"], color='#00008B', alpha=0.2)
ax3.set_ylabel('Motor Velocity (deg/s²)', color='#00008B')
ax3.set_ylim(get_symmetric_ylim(mean_baseline_df["Motor_Velocity_Baseline"], sem_baseline_df["Motor_Velocity_Baseline"]))
ax3.yaxis.label.set_color('#00008B')

# Running Velocity axis (Velocity_0X)
ax4 = ax.twinx()
ax4.spines['right'].set_position(('outward', 100))  
ax4.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0X_Baseline"]*1000, color='orange', linestyle='-', alpha=0.8)
ax4.fill_between(mean_baseline_df.index, (mean_baseline_df["Velocity_0X_Baseline"] - sem_baseline_df["Velocity_0X_Baseline"])*1000, 
                 (mean_baseline_df["Velocity_0X_Baseline"] + sem_baseline_df["Velocity_0X_Baseline"])*1000, color='orange', alpha=0.2)
ax4.set_ylabel('Running velocity (mm/s²) WRONG SCALE?', color='orange')
ax4.set_ylim(get_symmetric_ylim(mean_baseline_df["Velocity_0X_Baseline"]*1000, sem_baseline_df["Velocity_0X_Baseline"]*1000))
ax4.yaxis.label.set_color('orange')

# Turning Velocity axis (Velocity_0Y)
ax5 = ax.twinx()
ax5.spines['right'].set_position(('outward', 150))  
ax5.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0Y_Baseline"], color='#4682B4', linestyle='-', alpha=0.8)
ax5.fill_between(mean_baseline_df.index, mean_baseline_df["Velocity_0Y_Baseline"] - sem_baseline_df["Velocity_0Y_Baseline"], 
                 mean_baseline_df["Velocity_0Y_Baseline"] + sem_baseline_df["Velocity_0Y_Baseline"], color='#4682B4', alpha=0.2)
ax5.set_ylabel('Turning velocity (deg/s²) WRONG SCALE?', color='#4682B4')
ax5.set_ylim(get_symmetric_ylim(mean_baseline_df["Velocity_0Y_Baseline"], sem_baseline_df["Velocity_0Y_Baseline"]))
ax5.yaxis.label.set_color('#4682B4')

# ---- NEW: Plot Ellipse signals ----

# Ellipse.Diameter_eye1 axis
ax6 = ax.twinx()
ax6.spines['right'].set_position(('outward', 200))  
ax6.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], color='purple', linestyle='-', alpha=0.8)
ax6.fill_between(mean_baseline_df.index, mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] - sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
                mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] + sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], color='purple', alpha=0.2)
ax6.set_ylabel('Ellipse Diameter', color='purple')
ax6.set_ylim(get_symmetric_ylim(mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], sem_baseline_df["Ellipse.Diameter_eye1_Baseline"]))

# Ellipse.Center.X_eye1 axis
ax7 = ax.twinx()
ax7.spines['right'].set_position(('outward', 250))  
ax7.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], color='magenta', linestyle='-', alpha=0.8)
ax7.fill_between(mean_baseline_df.index, mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] - sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
                mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] + sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], color='magenta', alpha=0.2)
ax7.set_ylabel('Ellipse Center X', color='magenta')
ax7.set_ylim(get_symmetric_ylim(mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], sem_baseline_df["Ellipse.Center.X_eye1_Baseline"]))

# Add saccade events as markers
saccade_times = [saccade_onset_time, saccade_time, saccade_offset_time]
saccade_labels = ['Onset', 'Peak', 'Offset']
saccade_colors = ['blue', 'orange', 'red']

for saccade_time, label, color in zip(saccade_times, saccade_labels, saccade_colors):
    ax.axvline(x=(saccade_time - saccade_time).total_seconds(), color=color, linestyle='--', alpha=0.8, label=f'Saccade {label}')

# Add legend for saccade markers
ax.legend(loc='upper right')

# Adjust layout and show plot
fig.tight_layout()
plt.show()
# Save the figure as a PNG file at 1200 dpi
#fig.savefig(save_path / f"figure2_{event_name}.png", dpi=1200, bbox_inches='tight')
fig.savefig(save_path / f"figure2_{event_name}.pdf", dpi=1200, bbox_inches='tight')



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_selected_events(
    aligned_df,
    events_to_include,
    baseline_window,
    mouse_name,
    event_name,
    save_path,
    plot_width=14
):
    """
    Plots the mean & SEM of baseline-corrected signals for only the specified events.

    Parameters
    ----------
    aligned_df : pd.DataFrame
        DataFrame containing columns ["Time (s)", "Halt Time"] and the signals to be plotted.
    events_to_include : list of int
        List of specific event IDs (in "Halt Time") to include, e.g. [0, 1, 11, 16, 20].
    baseline_window : tuple
        A time window (start, end) in seconds for computing the baseline, e.g. (-1, 0).
    mouse_name : str
        Name of the mouse for the plot title.
    event_name : str
        Label for the event, used in the figure filename.
    save_path : Path or str
        Where to save the figure.
    plot_width : float
        Width of the plot in inches.
    """

    # 1) Subset data to only the events of interest
    subset_df = aligned_df[aligned_df["Halt Time"].isin(events_to_include)].copy()
    if subset_df.empty:
        print("No rows found for the specified events:", events_to_include)
        return

    # 2) Compute baseline values for each signal within the baseline_window
    #    *only* for the selected events
    baseline_mask = (
        (subset_df["Time (s)"] >= baseline_window[0]) &
        (subset_df["Time (s)"] <= baseline_window[1])
    )
    baseline_df = subset_df[baseline_mask].groupby("Halt Time").mean()

    # 3) Subtract baseline from each signal (except Photodiode)
    for signal_name in ["z_470", "z_560", "Motor_Velocity", "Velocity_0X", "Velocity_0Y"]:
        subset_df[f"{signal_name}_Baseline"] = (
            subset_df[signal_name] - 
            subset_df["Halt Time"].map(baseline_df[signal_name])
        )

    # (NEW) Baseline correction for Ellipse signals
    for signal_name in ["Ellipse.Diameter_eye1", "Ellipse.Center.X_eye1"]:
        subset_df[f"{signal_name}_Baseline"] = (
            subset_df[signal_name] -
            subset_df["Halt Time"].map(baseline_df[signal_name])
        )

    # 4) Compute mean and SEM for the baseline-adjusted signals
    mean_baseline_df = subset_df.groupby("Time (s)").mean()
    sem_baseline_df = subset_df.groupby("Time (s)").sem()

    # Helper function to get symmetrical y-limits around zero
    def get_symmetric_ylim(mean_data, sem_data):
        max_abs_value = max(
            abs(mean_data).max() + sem_data.max(),
            abs(mean_data).min() - sem_data.min()
        )
        return (-max_abs_value, max_abs_value)

    # 5) Create the figure
    fig, ax = plt.subplots(figsize=(plot_width, 6))

    # Photodiode
    ax.plot(mean_baseline_df.index, mean_baseline_df["Photodiode_int"], color='grey', alpha=0.8)
    ax.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Photodiode_int"] - sem_baseline_df["Photodiode_int"], 
        mean_baseline_df["Photodiode_int"] + sem_baseline_df["Photodiode_int"], 
        color='grey', alpha=0.2
    )
    ax.set_xlabel('Time (s) relative to halt')
    ax.set_ylabel('Photodiode', color='grey')
    ax.set_title(f'Baselined Mean & SEM of Selected Events - {mouse_name}')

    # Fluorescence axis (z_470 and z_560)
    ax2 = ax.twinx()
    ax2.plot(mean_baseline_df.index, mean_baseline_df["z_470_Baseline"], color='green', linestyle='-', alpha=0.8)
    ax2.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["z_470_Baseline"] - sem_baseline_df["z_470_Baseline"], 
        mean_baseline_df["z_470_Baseline"] + sem_baseline_df["z_470_Baseline"], 
        color='green', alpha=0.2
    )
    ax2.plot(mean_baseline_df.index, mean_baseline_df["z_560_Baseline"], color='red', linestyle='-', alpha=0.8)
    ax2.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["z_560_Baseline"] - sem_baseline_df["z_560_Baseline"], 
        mean_baseline_df["z_560_Baseline"] + sem_baseline_df["z_560_Baseline"], 
        color='red', alpha=0.2
    )
    ax2.set_ylabel('Fluorescence (z-score, red 560nm)', color='green')
    ax2.set_ylim(get_symmetric_ylim(
        pd.concat([mean_baseline_df["z_470_Baseline"], mean_baseline_df["z_560_Baseline"]]),
        pd.concat([sem_baseline_df["z_470_Baseline"], sem_baseline_df["z_560_Baseline"]])
    ))
    ax2.yaxis.label.set_color('green')

    # Motor Velocity axis
    ax3 = ax.twinx()
    ax3.spines['right'].set_position(('outward', 50))  
    ax3.plot(mean_baseline_df.index, mean_baseline_df["Motor_Velocity_Baseline"], color='#00008B', linestyle='-', alpha=0.8)
    ax3.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Motor_Velocity_Baseline"] - sem_baseline_df["Motor_Velocity_Baseline"], 
        mean_baseline_df["Motor_Velocity_Baseline"] + sem_baseline_df["Motor_Velocity_Baseline"], 
        color='#00008B', alpha=0.2
    )
    ax3.set_ylabel('Motor Velocity (deg/s²)', color='#00008B')
    ax3.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Motor_Velocity_Baseline"], 
        sem_baseline_df["Motor_Velocity_Baseline"]
    ))
    ax3.yaxis.label.set_color('#00008B')

    # Running Velocity axis (Velocity_0X)
    ax4 = ax.twinx()
    ax4.spines['right'].set_position(('outward', 100))  
    ax4.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0X_Baseline"]*1000, color='orange', linestyle='-', alpha=0.8)
    ax4.fill_between(
        mean_baseline_df.index,
        (mean_baseline_df["Velocity_0X_Baseline"] - sem_baseline_df["Velocity_0X_Baseline"])*1000, 
        (mean_baseline_df["Velocity_0X_Baseline"] + sem_baseline_df["Velocity_0X_Baseline"])*1000, 
        color='orange', alpha=0.2
    )
    ax4.set_ylabel('Running velocity (mm/s²) WRONG SCALE?', color='orange')
    ax4.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Velocity_0X_Baseline"]*1000, 
        sem_baseline_df["Velocity_0X_Baseline"]*1000
    ))
    ax4.yaxis.label.set_color('orange')

    # Turning Velocity axis (Velocity_0Y)
    ax5 = ax.twinx()
    ax5.spines['right'].set_position(('outward', 150))  
    ax5.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0Y_Baseline"], color='#4682B4', linestyle='-', alpha=0.8)
    ax5.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Velocity_0Y_Baseline"] - sem_baseline_df["Velocity_0Y_Baseline"], 
        mean_baseline_df["Velocity_0Y_Baseline"] + sem_baseline_df["Velocity_0Y_Baseline"], 
        color='#4682B4', alpha=0.2
    )
    ax5.set_ylabel('Turning velocity (deg/s²) WRONG SCALE?', color='#4682B4')
    ax5.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Velocity_0Y_Baseline"], 
        sem_baseline_df["Velocity_0Y_Baseline"] 
    ))
    ax5.yaxis.label.set_color('#4682B4')

    # Ellipse.Diameter_eye1 axis
    ax6 = ax.twinx()
    ax6.spines['right'].set_position(('outward', 200))  
    ax6.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], color='purple', linestyle='-', alpha=0.8)
    ax6.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] - sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
        mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] + sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
        color='purple', alpha=0.2
    )
    ax6.set_ylabel('Ellipse Diameter', color='purple')
    ax6.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
        sem_baseline_df["Ellipse.Diameter_eye1_Baseline"]
    ))

    # Ellipse.Center.X_eye1 axis
    ax7 = ax.twinx()
    ax7.spines['right'].set_position(('outward', 250))  
    ax7.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], color='magenta', linestyle='-', alpha=0.8)
    ax7.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] - sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
        mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] + sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
        color='magenta', alpha=0.2
    )
    ax7.set_ylabel('Ellipse Center X', color='magenta')
    ax7.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
        sem_baseline_df["Ellipse.Center.X_eye1_Baseline"]
    ))

    fig.tight_layout()
    plt.show()
    # Save the figure
    fig.savefig(save_path / f"figure2_{event_name}.png", dpi=1200, bbox_inches='tight')


# ----------------------------
# Example usage:
# ----------------------------
events_to_include = [0, 1, 11, 16, 20]
plot_selected_events(
    aligned_df=aligned_df,
    events_to_include=events_to_include,
    baseline_window=(-1, 0),  # example baseline window
    mouse_name="MouseA",
    event_name="MyEvents",
    save_path=Path("./output_plots"),
    plot_width=14
)


In [ ]:
# aligned_df.head()

In [ ]:
# import importlib #for force updating changed packages 
# # Force reload the modules
# importlib.reload(harp_resources.process)
# importlib.reload(harp_resources.utils)
# # Reassign after reloading to ensure updated references
# process = harp_resources.process
# utils = harp_resources.utils

In [ ]:
# pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)
# mouse_name = "B6J2717"